# 05 Inheritance - Exercises

## Demo

Class `Person` has a single data attr `age`.  
`age` is encapsulated in a property and its value must be within an allowable range 0-150.  
Class `Employee` is a `Person`. We want to use diff age range - 18-65.  

**How can this be done?**

In [3]:
class Person:
    min_age = 0
    max_age = 150

    def __init__(self, age):
        self.age = age

    @staticmethod
    def __validate_age(age):
        if not (Person.min_age <= age <= Person.max_age):
            raise ValueError(f'Age mush be between {Person.min_age} and {Person.max_age}')

    @property
    def age(self):
        return self.__age

    @age.setter
    def age(self, new_age):
        self.__validate_age(new_age)
        self.__age = new_age
        
        
p1 = Person(24)
# p2 = Person(-3)  # ValueError
p1

### Solution 1

Create new validation method in `Employee`.  
The new method, has a new name. To use it, it needs to be called in a new setter method.  
A setter requires a getter. Thus, a whole property is required.  

**This is a lot of repeated code - not good solution.**

In [4]:
class Employee(Person):
    __MIN_EMPLOYEE_AGE = 18
    __MAX_EMPLOYEE_AGE = 65

    def __init__(self, age, company):
        # From the parent class, get the init method and call it with argument age..
        super().__init__(age)
        # Set the company attr to the value of the company param.
        self.company = company

    @staticmethod
    def __validate_employee_age(age):
        if age < Employee.__MIN_EMPLOYEE_AGE or Employee.__MAX_EMPLOYEE_AGE < age:
            raise ValueError(f'Age mush be between {Employee.__MIN_EMPLOYEE_AGE} and {Employee.__MAX_EMPLOYEE_AGE}')

    @property
    def age(self):
        return self.__age

    @age.setter
    def age(self, new_age):
        self.__validate_employee_age(new_age)
        self.__age = new_age


barry = Employee(33, 'IBM')
# garry = Employee(75, 'IBM')  # ValueError

### Solution 2 (not working)

Duplicate the validation method - create method in `Employee` with the same name.  
This will not work:  
- the validation method is called in the setter of the parent class
- however, the validation methods are private, meaning their names are mangled
- therefore, the validation in `Person` will be called every time

In [5]:
class Employee(Person):
    __MIN_EMPLOYEE_AGE = 18
    __MAX_EMPLOYEE_AGE = 65

    def __init__(self, age, company):
        super().__init__(age)
        self.company = company

    @staticmethod
    def __validate_age(age):
        if age < Employee.__MIN_EMPLOYEE_AGE or Employee.__MAX_EMPLOYEE_AGE < age:
            raise ValueError(f'Age mush be between {Employee.__MIN_EMPLOYEE_AGE} and {Employee.__MAX_EMPLOYEE_AGE}')
            
    
barry = Employee(33, 'IBM')
garry = Employee(75, 'IBM')  # Incorrectly, no exceptions are raised.

### Solution 3 (the best one)

Change the validation method from `staticmethod` to `classmethod`.  
This way, calling the method from a child class will use the correct age limits.  
Important: the class attrs that specify the allowable limits (`min_age`, `max_age`) must be public. If private (hidden), the names will be mangled.1 

In [8]:
class Person:
    min_age = 0
    max_age = 150

    def __init__(self, age):
        self.age = age

    @classmethod
    def __validate_age(cls, age):
        if age < cls.min_age or cls.max_age < age:
            raise ValueError(f'Age mush be between {cls.min_age} and {cls.max_age}')

    @property
    def age(self):
        return self.__age

    @age.setter
    def age(self, new_age):
        self.__validate_age(new_age)
        self.__age = new_age


class Employee(Person):
    min_age = 18
    max_age = 65

    def __init__(self, age, company):
        super().__init__(age)
        self.company = company

        
p1 = Person(90)  # OK
p2 = Employee(36, '7/11')  # OK
# p3 = Employee(70, 'Billa')  # ValueError

## Method resolution order

In [18]:
class ElectricalDevice:
    def __init__(self, name, voltage_rating):
        self.name = name
        self.voltage_rating = voltage_rating

    def __repr__(self):
        # Self-documenting expression - `{self.voltage_rating=}`.
        return f'{self.name:<20}, {self.voltage_rating=}'
    
    
water_heater = ElectricalDevice('Water heater', 230)
mobile = ElectricalDevice('Mobile', 5)

print(water_heater)
print(mobile)

Water heater        , self.voltage_rating=230
Mobile              , self.voltage_rating=5


In [21]:
class AudioDevice:
    def __init__(self, name, db):
        self.name = name
        self.db = db

    def __repr__(self):
        return f'{self.name:<20}, {self.db}'
    

radio = AudioDevice('Radio', 40)
radio

Radio               , 40

In [33]:
class Speakers(ElectricalDevice, AudioDevice):
    def __init__(self, name, voltage_rating, db):
        # super().__init__(name, voltage_rating)  # super refers ONLY to the first parent
        ElectricalDevice.__init__(self, name, voltage_rating)
        AudioDevice.__init__(self, name, db)

    def __repr__(self):
        return f'{ElectricalDevice.__repr__(self)}\n{AudioDevice.__repr__(self)}'


marshal = Speakers('Marshal SV1000', 230, 90)
print(marshal)

Marshal SV1000      , self.voltage_rating=230
Marshal SV1000      , 90


In [34]:
# Classes have mro, objs do not!

# marshal.mro()  # AttributeError: 'Speakers' object has no attribute 'mro'
Speakers.mro()

[__main__.Speakers, __main__.ElectricalDevice, __main__.AudioDevice, object]

In [32]:
print(help(Speakers))

Help on class Speakers in module __main__:

class Speakers(ElectricalDevice, AudioDevice)
 |  Speakers(name, voltage_rating, db)
 |  
 |  Method resolution order:
 |      Speakers
 |      ElectricalDevice
 |      AudioDevice
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, voltage_rating, db)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from ElectricalDevice:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

None
